# Requirments
- Python 3.11.9
- install the latest backtesting.py version `pip install git+https://github.com/kernc/backtesting.py.git`
- install additional deps `pip install yfinance pandas ipykernal`
    - you only need ipykernel if you are running a jupyter notebook

In [2]:
import yfinance as yf
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import pandas as pd

# import numpy as np
# print(np.__version__)

%load_ext autoreload
%autoreload 2

In [3]:
class EMACrossStrategy(Strategy):
    n1 = 20  # fast EMA period
    n2 = 50  # slow EMA period

    def init(self):
        # Calculate EMAs
        self.ema1 = self.I(
            lambda x: pd.Series(x).ewm(span=self.n1, adjust=False).mean(),
            self.data.Close,
        )
        self.ema2 = self.I(
            lambda x: pd.Series(x).ewm(span=self.n2, adjust=False).mean(),
            self.data.Close,
        )

    def next(self):
        if crossover(self.ema1, self.ema2):
            self.buy()
        elif crossover(self.ema2, self.ema1):
            self.sell()

In [4]:
# Download the Data
start_date = "2023-08-12"
end_date = "2024-08-11"
data = yf.download("BTC-USD", start=start_date, end=end_date)
data

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-08-12,29399.787109,29465.113281,29357.587891,29415.964844,29415.964844,6194358008
2023-08-13,29416.593750,29441.433594,29265.806641,29282.914062,29282.914062,7329897180
2023-08-14,29283.263672,29660.253906,29124.105469,29408.443359,29408.443359,14013695304
2023-08-15,29408.048828,29439.121094,29088.853516,29170.347656,29170.347656,12640195779
2023-08-16,29169.074219,29221.976562,28701.779297,28701.779297,28701.779297,14949271904
...,...,...,...,...,...,...
2024-08-06,53991.347656,57059.917969,53973.273438,56034.316406,56034.316406,49300484106
2024-08-07,56040.632812,57726.882812,54620.507812,55027.460938,55027.460938,41637562185
2024-08-08,55030.027344,62673.765625,54766.726562,61710.136719,61710.136719,45298472567


In [6]:
# Create and run the backtest
bt = Backtest(data, EMACrossStrategy, cash=100000, commission=0.002)
stats = bt.run()

# Print results
print(stats)
# Plot the result
bt.plot()

Start                     2023-08-12 00:00:00
End                       2024-08-10 00:00:00
Duration                    364 days 00:00:00
Exposure Time [%]                   84.657534
Equity Final [$]                198635.666969
Equity Peak [$]                 235242.475563
Return [%]                          98.635667
Buy & Hold Return [%]              107.186175
Return (Ann.) [%]                   98.635667
Volatility (Ann.) [%]               93.601267
Sharpe Ratio                         1.053786
Sortino Ratio                        3.562058
Calmar Ratio                         4.051129
Max. Drawdown [%]                  -24.347699
Avg. Drawdown [%]                   -4.429224
Max. Drawdown Duration      150 days 00:00:00
Avg. Drawdown Duration       16 days 00:00:00
# Trades                                    1
Win Rate [%]                            100.0
Best Trade [%]                     117.412197
Worst Trade [%]                    117.412197
Avg. Trade [%]                    

/Users/aquawolf/Projects/live-stream-projects/algo-trading/backtesting-crypto/.venv/lib/python3.11/site-packages/backtesting/_plotting.py:456: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample(resample_rule, label='left')


GridPlot(id='p1705', ...)